In [93]:
!pip install -q -U google-generativeai python-dotenv pandas Jinja2

I0000 00:00:1722553394.568100 6997231 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [184]:
import google.generativeai as genai
from IPython.display import Markdown, Image
import os, json
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [259]:
def get_predicted_data(image_path):
    # Upload the file
    sample_file = genai.upload_file(path=image_path)

    # Run Gemini model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = """
    As an expert nutritionist, analyze this food image taken 50cm above the dish. Provide accurate nutrition information in the following steps:

    1. Identify all visible food items and ingredients.
    2. For each item, provide per gram nutritional data (calories, protein, carbs, fat) based on the USDA Food and Nutrient Database.
    3. Estimate the weight in grams for each item, considering the 50cm camera distance and visual cues.
    4. Calculate total nutritional content by multiplying per gram values with estimated weights.
    5. Consider preparation methods and visible condiments in your analysis.
    6. Round all estimates to the nearest 100th decimal place.

    Output your analysis in this JSON format, do not output anything else:
    {
        "items": [
            {
                "item": "<food_item>",
                "estimated_weight_g": <weight>,
                "per_100g": {
                    "calories": <calories>,
                    "protein": <protein>,
                    "carb": <carb>,
                    "fat": <fat>
                },
                "total": {
                    "calories": <calories>,
                    "protein": <protein>,
                    "carb": <carb>,
                    "fat": <fat>
                }
            },
            // Repeat for each food item
        ]
    }

    Prioritize accuracy in your estimates, using all available visual information and your expertise.
    """
    response = model.generate_content([sample_file, prompt])

    return response.text

In [252]:
def ensure_json_output(response_text):
    try:
        # Attempt to parse the response as JSON
        json_data = json.loads(response_text)
        return json.dumps(json_data)  # Return a properly formatted JSON string
    except json.JSONDecodeError:
        # If parsing fails, attempt to extract JSON from the text
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        if json_start != -1 and json_end != -1:
            json_string = response_text[json_start:json_end]
            try:
                json_data = json.loads(json_string)
                return json.dumps(json_data)  # Return a properly formatted JSON string
            except json.JSONDecodeError:
                pass
        
        # If extraction fails, return a JSON-formatted error message
        return json.dumps({
            "error": "Unable to parse response as JSON",
            "raw_response": response_text
        })


In [271]:
def format_predicted_data(predicted_data, image_path):
    predicted_data = ensure_json_output(predicted_data)
    with open('actual_data.json', 'r') as file:
        actual_data = json.load(file)

    # Ensure predicted_data is a dictionary
    if isinstance(predicted_data, str):
        try:
            predicted_data = json.loads(predicted_data)
        except json.JSONDecodeError:
            # If JSON parsing fails, return an error message
            return {
                "items": pd.DataFrame(),
                "summary": pd.DataFrame([{
                    "Nutrient": nutrient,
                    "Actual": actual_data[image_path][nutrient.lower()],
                    "Predicted": "Error",
                    "Percent Error": "N/A"
                } for nutrient in ["Calories", "Protein", "Carb", "Fat"]])
            }

    # Create items DataFrame
    items_df = pd.DataFrame([
        {
            "Item": item["item"],
            "Estimated Weight (g)": item["estimated_weight_g"],
            "Calories": f"{item['total']['calories']} ({item['per_100g']['calories']}/100g)",
            "Protein (g)": f"{item['total']['protein']} ({item['per_100g']['protein']}/100g)",
            "Carbs (g)": f"{item['total']['carb']} ({item['per_100g']['carb']}/100g)",
            "Fat (g)": f"{item['total']['fat']} ({item['per_100g']['fat']}/100g)"
        }
        for item in predicted_data["items"]
    ])

    # Calculate totals
    totals = items_df[["Calories", "Protein (g)", "Carbs (g)", "Fat (g)"]].apply(lambda x: x.str.split().str[0].astype(float)).sum()

    # Create summary DataFrame
    summary_data = []
    for property, actual in actual_data[image_path].items():
        if property.lower() == "protein":
            predicted = totals["Protein (g)"]
        elif property.lower() == "carb":
            predicted = totals["Carbs (g)"]
        elif property.lower() == "fat":
            predicted = totals["Fat (g)"]
        else:
            predicted = totals[property.capitalize()]
        
        percent_error = ((predicted - actual) / actual) * 100
        summary_data.append({
            "Nutrient": property.capitalize(),
            "Actual": round(actual, 2),
            "Predicted": round(predicted, 2),
            "Percent Error": round(percent_error, 2)
        })

    summary_df = pd.DataFrame(summary_data)

    # Style the DataFrames
    items_styled = items_df.style.set_caption("Food Items Breakdown").format({
        "Estimated Weight (g)": "{:.0f}",
        "Calories": "{}",
        "Protein (g)": "{}",
        "Carbs (g)": "{}",
        "Fat (g)": "{}"
    })

    def color_percent_error(val):
        color = 'red' if val < 0 else 'green'
        return f'color: {color}'

    summary_styled = summary_df.style.set_caption("Cumulative Nutritional Summary").format({
        "Actual": "{:.2f}",
        "Predicted": "{:.2f}",
        "Percent Error": "{:.2f}%"
    }).bar(subset=["Percent Error"], color=["red", "green"], vmin=-100, vmax=100).map(lambda x: 'color: white', subset=["Percent Error"])

    return {
        "items": items_styled,
        "summary": summary_styled
    }

In [272]:
predicted_data_1 = get_predicted_data("images/1.png")
formatted_predicted_data_1 = format_predicted_data(predicted_data_1, "images/1.png")
display(formatted_predicted_data_1["items"])
display(formatted_predicted_data_1["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,"Bacon, cooked",35,143.83 (410.95/100g),9.5 (27.13/100g),0.38 (1.08/100g),11.32 (32.33/100g)
1,"Egg, scrambled, cooked",100,143.36 (143.36/100g),12.56 (12.56/100g),1.98 (1.98/100g),9.51 (9.51/100g)
2,"Strawberries, raw",100,32.05 (32.05/100g),0.67 (0.67/100g),7.68 (7.68/100g),0.3 (0.3/100g)
3,"Blueberries, raw",50,28.52 (57.04/100g),0.37 (0.74/100g),7.25 (14.49/100g),0.17 (0.33/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,357.81,347.76,-2.81%
1,Protein,20.00,23.10,15.50%
2,Carb,20.96,17.29,-17.51%
3,Fat,21.69,21.30,-1.80%


In [127]:
predicted_data_2 = get_predicted_data("images/2.png")
formatted_predicted_data_2 = format_predicted_data(predicted_data_2, "images/2.png")
display(formatted_predicted_data_2["items"])
display(formatted_predicted_data_2["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Roast Chicken Breast,300,570 (190/100g),90 (30/100g),0 (0/100g),24 (8/100g)
1,Lettuce,100,15 (15/100g),1 (1/100g),3 (3/100g),0 (0/100g)
2,Tomato,100,18 (18/100g),1 (1/100g),4 (4/100g),0 (0/100g)
3,Carrots,150,62 (41/100g),2 (1/100g),15 (10/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,655.99,665.00,1.37%
1,Protein,93.24,94.00,0.82%
2,Carb,23.34,22.00,5.74%
3,Fat,20.22,24.00,18.69%


In [128]:
predicted_data_3 = get_predicted_data("images/3.png")
formatted_predicted_data_3 = format_predicted_data(predicted_data_3, "images/3.png")
display(formatted_predicted_data_3["items"])
display(formatted_predicted_data_3["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Watermelon,200,60 (30/100g),2 (1/100g),16 (8/100g),0 (0/100g)
1,Roasted Sweet Potato,100,90 (90/100g),2 (2/100g),20 (20/100g),0 (0/100g)
2,Blackberries,50,22 (43/100g),1 (1/100g),5 (10/100g),1 (1/100g)
3,Raspberries,50,26 (52/100g),1 (1/100g),6 (12/100g),1 (1/100g)
4,Pineapple,100,50 (50/100g),1 (1/100g),13 (13/100g),0 (0/100g)
5,Cottage Cheese (Low-fat),150,120 (80/100g),18 (12/100g),5 (3/100g),3 (2/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,351.89,368.00,4.58%
1,Protein,12.10,25.00,106.61%
2,Carb,57.50,65.00,13.04%
3,Fat,9.77,5.00,48.82%


In [129]:
predicted_data_4 = get_predicted_data("images/4.png")
formatted_predicted_data_4 = format_predicted_data(predicted_data_4, "images/4.png")
display(formatted_predicted_data_4["items"])
display(formatted_predicted_data_4["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,"Chicken Breast, Cooked, Roasted, or Baked (Skin Not Eaten)",150,248 (165/100g),47 (31/100g),0 (0/100g),6 (4/100g)
1,"Lettuce, Green Leaf, Raw",100,15 (15/100g),1 (1/100g),3 (3/100g),0 (0/100g)
2,"Onions, Red, Raw",50,20 (40/100g),1 (1/100g),5 (9/100g),0 (0/100g)
3,"Avocado, Raw, California",50,80 (160/100g),1 (2/100g),5 (9/100g),8 (15/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,184.99,363.00,96.23%
1,Protein,24.54,50.00,103.75%
2,Carb,8.90,13.00,46.07%
3,Fat,6.14,14.00,128.01%


In [273]:
predicted_data_4 = get_predicted_data("images/4.png")
formatted_predicted_data_4 = format_predicted_data(predicted_data_4, "images/4.png")
display(formatted_predicted_data_4["items"])
display(formatted_predicted_data_4["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,"Chicken breast, cooked, roasted",150,247.5 (165.0/100g),46.53 (31.02/100g),0.0 (0.0/100g),5.36 (3.57/100g)
1,"Lettuce, green leaf, raw",75,11.25 (15.0/100g),1.02 (1.36/100g),2.15 (2.87/100g),0.15 (0.2/100g)
2,"Onions, red, raw",50,20.0 (40.0/100g),0.55 (1.1/100g),4.67 (9.34/100g),0.05 (0.1/100g)
3,"Avocado, raw, california",50,80.0 (160.0/100g),1.0 (2.0/100g),4.27 (8.53/100g),7.33 (14.66/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,184.99,358.75,93.93%
1,Protein,24.54,49.10,100.08%
2,Carb,8.90,11.09,24.61%
3,Fat,6.14,12.89,109.93%


In [131]:
predicted_data_5 = get_predicted_data("images/5.png")
formatted_predicted_data_5 = format_predicted_data(predicted_data_5, "images/5.png")
display(formatted_predicted_data_5["items"])
display(formatted_predicted_data_5["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Cantaloupe,120,41 (34/100g),1 (1/100g),10 (8/100g),0 (0/100g)
1,Bacon,30,120 (400/100g),5 (17/100g),0 (1/100g),11 (37/100g)
2,"Potatoes, roasted",80,74 (93/100g),2 (2/100g),17 (21/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,139.56,235.00,68.39%
1,Protein,5.40,8.00,48.15%
2,Carb,14.66,27.00,84.17%
3,Fat,6.64,11.00,65.66%


In [274]:
predicted_data_5 = get_predicted_data("images/5.png")
formatted_predicted_data_5 = format_predicted_data(predicted_data_5, "images/5.png")
display(formatted_predicted_data_5["items"])
display(formatted_predicted_data_5["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Cantaloupe,100,34.0 (34.0/100g),0.8 (0.8/100g),8.16 (8.16/100g),0.19 (0.19/100g)
1,"Potato (baked, without skin)",75,69.75 (93.0/100g),1.88 (2.5/100g),15.1 (20.13/100g),0.13 (0.17/100g)
2,"Bacon (cooked, pan-fried)",30,122.4 (408.0/100g),3.61 (12.03/100g),0.32 (1.08/100g),11.2 (37.33/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,139.56,226.15,62.04%
1,Protein,5.40,6.29,16.48%
2,Carb,14.66,23.58,60.85%
3,Fat,6.64,11.52,73.49%


In [132]:
predicted_data_6 = get_predicted_data("images/6.png")
formatted_predicted_data_6 = format_predicted_data(predicted_data_6, "images/6.png")
display(formatted_predicted_data_6["items"])
display(formatted_predicted_data_6["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Cooked Penne Pasta,100,131 (131/100g),5 (5/100g),27 (27/100g),1 (1/100g)
1,Cooked Beef Cubes,100,250 (250/100g),26 (26/100g),0 (0/100g),17 (17/100g)
2,Cooked Spinach,50,12 (23/100g),2 (3/100g),2 (4/100g),0 (0/100g)
3,Olive Oil (Assumed for Cooking),10,88 (884/100g),0 (0/100g),0 (0/100g),10 (100/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,950.40,481.00,49.39%
1,Protein,74.77,33.00,55.86%
2,Carb,54.13,29.00,46.43%
3,Fat,45.30,28.00,38.19%


In [265]:
predicted_data_6 = get_predicted_data("images/6.png")
formatted_predicted_data_6 = format_predicted_data(predicted_data_6, "images/6.png")
display(formatted_predicted_data_6["items"])
display(formatted_predicted_data_6["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Cooked Penne Pasta,200,262.0 (131.0/100g),9.6 (4.8/100g),53.4 (26.7/100g),2.4 (1.2/100g)
1,"Cooked Beef, bottom round roast, lean only",150,297.0 (198.0/100g),42.9 (28.6/100g),0.0 (0.0/100g),12.75 (8.5/100g)
2,"Spinach, cooked, no salt added",50,11.5 (23.0/100g),1.45 (2.9/100g),1.8 (3.6/100g),0.2 (0.4/100g)
3,Olive Oil,10,88.4 (884.0/100g),0.0 (0.0/100g),0.0 (0.0/100g),10.0 (100.0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,950.40,658.90,30.67%
1,Protein,74.77,53.95,27.85%
2,Carb,54.13,55.20,1.98%
3,Fat,45.30,25.35,44.04%


In [134]:
predicted_data_7 = get_predicted_data("images/7.png")
formatted_predicted_data_7 = format_predicted_data(predicted_data_7, "images/7.png")
display(formatted_predicted_data_7["items"])
display(formatted_predicted_data_7["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Scrambled eggs with cheese,70,140 (200/100g),12 (17/100g),1 (2/100g),11 (15/100g)
1,Fried potatoes (hash browns),100,150 (150/100g),2 (2/100g),20 (20/100g),8 (8/100g)
2,Cooked bacon,20,80 (400/100g),5 (27/100g),0 (1/100g),7 (35/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,284.69,370.00,29.97%
1,Protein,13.84,19.00,37.28%
2,Carb,14.42,21.00,45.63%
3,Fat,18.80,26.00,38.30%


In [275]:
predicted_data_7 = get_predicted_data("images/7.png")
formatted_predicted_data_7 = format_predicted_data(predicted_data_7, "images/7.png")
display(formatted_predicted_data_7["items"])
display(formatted_predicted_data_7["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Scrambled eggs with cheddar cheese,100,207.0 (207.0/100g),14.69 (14.69/100g),2.78 (2.78/100g),15.54 (15.54/100g)
1,Cooked bacon,30,122.4 (408.0/100g),8.11 (27.03/100g),0.32 (1.08/100g),9.75 (32.51/100g)
2,Hash browns,100,149.0 (149.0/100g),2.0 (2.0/100g),20.0 (20.0/100g),7.0 (7.0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,284.69,478.40,68.04%
1,Protein,13.84,24.80,79.19%
2,Carb,14.42,23.10,60.19%
3,Fat,18.80,32.29,71.76%


In [205]:
predicted_data_8 = get_predicted_data("images/8.png")

In [206]:
formatted_predicted_data_8 = format_predicted_data(predicted_data_8, "images/8.png")
display(formatted_predicted_data_8["items"])
display(formatted_predicted_data_8["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Bacon,45,243 (541/100g),17 (37/100g),0 (1/100g),19 (42/100g)
1,Almonds,30,172 (574/100g),6 (21/100g),7 (22/100g),15 (50/100g)
2,Blackberries,20,9 (43/100g),0 (1/100g),2 (10/100g),0 (1/100g)
3,Lettuce (red leaf),30,5 (15/100g),0 (1/100g),1 (3/100g),0 (0/100g)
4,Raspberries,10,5 (53/100g),0 (1/100g),1 (12/100g),0 (1/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,557.73,434.00,22.18%
1,Protein,25.77,23.00,10.75%
2,Carb,23.81,11.00,53.80%
3,Fat,42.54,34.00,20.08%


In [138]:
predicted_data_9 = get_predicted_data("images/9.png")
formatted_predicted_data_9 = format_predicted_data(predicted_data_9, "images/9.png")
display(formatted_predicted_data_9["items"])
display(formatted_predicted_data_9["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Roasted Brussels Sprouts,150,57 (38/100g),4 (3/100g),12 (8/100g),0 (0/100g)
1,Mixed Green Salad,150,30 (20/100g),1 (1/100g),6 (4/100g),0 (0/100g)
2,Brown Rice,150,166 (111/100g),7 (5/100g),34 (23/100g),1 (1/100g)
3,Roasted Pork,150,319 (213/100g),30 (20/100g),0 (0/100g),21 (14/100g)
4,Cauliflower Rice,50,12 (25/100g),1 (2/100g),2 (5/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,583.38,584.00,0.11%
1,Protein,42.29,43.00,1.68%
2,Carb,52.42,54.00,3.01%
3,Fat,23.33,22.00,5.70%


In [139]:
predicted_data_10 = get_predicted_data("images/10.png")
formatted_predicted_data_10 = format_predicted_data(predicted_data_10, "images/10.png")
display(formatted_predicted_data_10["items"])
display(formatted_predicted_data_10["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Broccoli,100,34 (34/100g),3 (3/100g),7 (7/100g),0 (0/100g)
1,"Sweet Potato, cooked, roasted, without skin, no salt",150,135 (90/100g),3 (2/100g),30 (20/100g),0 (0/100g)
2,"Blackberries, raw",50,22 (43/100g),1 (1/100g),5 (10/100g),1 (1/100g)
3,"Raspberries, raw",50,26 (52/100g),1 (1/100g),6 (12/100g),1 (1/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,163.86,217.00,32.43%
1,Protein,2.51,8.00,218.73%
2,Carb,28.49,48.00,68.48%
3,Fat,5.30,2.00,62.26%


In [148]:
predicted_data_11 = get_predicted_data("images/11.png")
formatted_predicted_data_11 = format_predicted_data(predicted_data_11, "images/11.png")
display(formatted_predicted_data_11["items"])
display(formatted_predicted_data_11["summary"])

I0000 00:00:1722555582.509106 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722555585.738187 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Beef jerky,60,165 (275/100g),20 (33/100g),6 (10/100g),8 (13/100g)
1,Spinach,50,12 (23/100g),2 (3/100g),2 (4/100g),0 (0/100g)
2,Carrots,20,8 (41/100g),0 (1/100g),2 (10/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,428.67,185.00,56.84%
1,Protein,24.47,22.00,10.09%
2,Carb,4.90,10.00,104.08%
3,Fat,35.14,8.00,77.23%


In [254]:
predicted_data_11 = get_predicted_data("images/11.png")
formatted_predicted_data_11 = format_predicted_data(predicted_data_11, "images/11.png")
display(formatted_predicted_data_11["items"])
display(formatted_predicted_data_11["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,"Beef, brisket, flat half, separable lean and fat, trimmed to 0"" fat, all grades, cooked, braised",150,417.0 (278.0/100g),54.74 (36.49/100g),0.0 (0.0/100g),20.82 (13.88/100g)
1,"Spinach, raw",50,11.5 (23.0/100g),1.43 (2.86/100g),1.82 (3.63/100g),0.2 (0.4/100g)
2,"Carrots, raw",25,10.25 (41.0/100g),0.23 (0.93/100g),2.4 (9.58/100g),0.06 (0.24/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,428.67,438.75,2.35%
1,Protein,24.47,56.40,130.49%
2,Carb,4.90,4.22,13.88%
3,Fat,35.14,21.08,40.01%


In [240]:
predicted_data_12 = get_predicted_data("images/12.png")

In [248]:
formatted_predicted_data_12 = format_predicted_data(predicted_data_12, "images/12.png")
display(formatted_predicted_data_12["items"])
display(formatted_predicted_data_12["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,"Cooked Chicken Breast, diced",250,412.5 (165.0/100g),77.5 (31.0/100g),0.0 (0.0/100g),9.0 (3.6/100g)
1,"Plain Yogurt, nonfat",100,56.0 (56.0/100g),10.0 (10.0/100g),5.0 (5.0/100g),0.4 (0.4/100g)
2,"Raw Carrots, shredded",50,20.5 (41.0/100g),0.45 (0.9/100g),4.8 (9.6/100g),0.1 (0.2/100g)
3,"Red Onion, raw, chopped",25,10.0 (40.0/100g),0.28 (1.1/100g),2.34 (9.34/100g),0.03 (0.1/100g)
4,"Cherry Tomatoes, halved",50,8.0 (16.0/100g),0.44 (0.88/100g),1.95 (3.89/100g),0.1 (0.2/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,506.36,507.00,0.13%
1,Protein,77.39,88.67,14.58%
2,Carb,16.41,14.09,14.14%
3,Fat,13.48,9.63,28.56%
